<a href="https://colab.research.google.com/github/GaetanAm/Master-1-Machine-Learning/blob/main/Exercice_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EXERCICE 4

Étape 1 : Charger le dataset

In [3]:
# Importer les bibliothèques nécessaires
from google.colab import files
import pandas as pd

# Télécharger le fichier localement
uploaded = files.upload()


Saving creditcard.csv to creditcard.csv


In [4]:
# Charger le dataset à partir du fichier téléchargé
data = pd.read_csv(list(uploaded.keys())[0])

# Vérifier les premières lignes
print("Aperçu du dataset :")
print(data.head())

# Vérifier les valeurs manquantes
print("\nValeurs manquantes dans chaque colonne :")
print(data.isnull().sum())

Aperçu du dataset :
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26 

In [7]:
# Vérifier les dimensions et la répartition des classes
print("\nDimensions des données :", data.shape)
print("Répartition des classes :")
print(data['Class'].value_counts())


Dimensions des données : (284807, 31)
Répartition des classes :
Class
0    284315
1       492
Name: count, dtype: int64


Étape 2 : Préparer les données

Définir les variables indépendantes (X) et dépendantes (y) :

1.   Les colonnes V1 à V28 et Amount seront utilisées comme caractéristiques (X).
2.   La colonne Class sera la cible (y).



Créer les variables X et y :

In [8]:
# Séparer les caractéristiques et la cible
X = data.drop(columns=['Class'])
y = data['Class']

print("Dimensions de X :", X.shape)
print("Dimensions de y :", y.shape)


Dimensions de X : (284807, 30)
Dimensions de y : (284807,)


Étape 3 : Diviser en ensembles d'entraînement et de test
On Utilise train_test_split de sklearn pour diviser les données en deux ensembles :


1.   Entraînement (train) : pour ajuster le modèle.
2.   Test : pour évaluer les performances du modèle.



In [9]:
from sklearn.model_selection import train_test_split

# Diviser les données en train et test avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Vérifier les dimensions
print("Dimensions de X_train :", X_train.shape)
print("Dimensions de X_test :", X_test.shape)
print("Répartition des classes dans y_train :")
print(y_train.value_counts(normalize=True))
print("Répartition des classes dans y_test :")
print(y_test.value_counts(normalize=True))


Dimensions de X_train : (227845, 30)
Dimensions de X_test : (56962, 30)
Répartition des classes dans y_train :
Class
0    0.998271
1    0.001729
Name: proportion, dtype: float64
Répartition des classes dans y_test :
Class
0    0.99828
1    0.00172
Name: proportion, dtype: float64


Étape 4 : Standardiser les données

Pour améliorer la convergence des modèles (comme la régression logistique), on standardise les données en mettant toutes les colonnes sur une échelle comparable.

In [10]:
from sklearn.preprocessing import StandardScaler

# Initialiser le scaler
scaler = StandardScaler()

# Appliquer la standardisation sur les ensembles d'entraînement et de test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Standardisation terminée.")


Standardisation terminée.


Étape 5 : Appliquer SMOTE

Nous utilisons la bibliothèque imblearn pour appliquer SMOTE sur les données d'entraînement.

In [11]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Appliquer SMOTE à l'ensemble d'entraînement
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Vérifier la répartition des classes après SMOTE
print("Répartition des classes après SMOTE :")
print(Counter(y_train_smote))


Répartition des classes après SMOTE :
Counter({0: 227451, 1: 227451})


Étape 6 : Entraîner et évaluer le modèle

Nous allons entraîner un modèle de régression logistique sur les données équilibrées avec SMOTE et le comparer à un modèle entraîné sur les données originales.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Modèle sur les données originales
model_original = LogisticRegression(max_iter=200, random_state=42)
model_original.fit(X_train_scaled, y_train)
y_pred_original = model_original.predict(X_test_scaled)

print("\nRapport de classification (données originales) :")
print(classification_report(y_test, y_pred_original))



Rapport de classification (données originales) :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.63      0.72        98

    accuracy                           1.00     56962
   macro avg       0.91      0.82      0.86     56962
weighted avg       1.00      1.00      1.00     56962



In [13]:
# Modèle sur les données SMOTE
model_smote = LogisticRegression(max_iter=200, random_state=42)
model_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = model_smote.predict(X_test_scaled)

print("\nRapport de classification (données SMOTE) :")
print(classification_report(y_test, y_pred_smote))



Rapport de classification (données SMOTE) :
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.97     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.97      0.99     56962



### Étape 8 : Analyse et conclusion pour l'exercice 4

D'après les résultats obtenus, voici une analyse détaillée pour conclure l'exercice :

---

#### Comparaison des résultats

| **Modèle**           | **Classe** | **Précision** | **Rappel** | **F1-score** | **Support** |
|-----------------------|------------|---------------|------------|--------------|-------------|
| Données originales    | 0 (légitime) | **1.00**      | **1.00**   | **1.00**     | 56864       |
|                       | 1 (fraude)   | 0.83          | 0.63       | 0.72         | 98          |
| **Moyenne macro**     | -           | 0.91          | 0.82       | 0.86         | 56962       |
| **Accuracy globale**  | -           | **1.00**      | -          | -            | 56962       |
| Données SMOTE         | 0 (légitime) | **1.00**      | 0.97       | 0.98         | 56864       |
|                       | 1 (fraude)   | 0.06          | **0.92**   | 0.11         | 98          |
| **Moyenne macro**     | -           | 0.53          | **0.95**   | 0.55         | 56962       |
| **Accuracy globale**  | -           | 0.97          | -          | -            | 56962       |

---

#### Analyse des résultats

1. **Modèle avec les données originales :**
   - Le modèle a une précision parfaite pour les transactions légitimes (**1.00**) mais une performance modérée pour les fraudes.
   - Le rappel de 0.63 pour les fraudes montre que 37 % des fraudes ne sont pas détectées.
   - Ce modèle est biaisé en faveur de la classe majoritaire (transactions légitimes).

2. **Modèle avec les données SMOTE :**
   - Le rappel pour les fraudes est nettement amélioré (**0.92**), détectant presque toutes les fraudes.
   - La précision pour les fraudes est très faible (**0.06**), ce qui signifie un grand nombre de faux positifs.
   - Bien que le rappel pour les transactions légitimes reste élevé (**0.97**), la précision générale est impactée.

---

#### Étape 9 : Conclusion finale

- **Amélioration avec SMOTE** :
  - L'utilisation de SMOTE améliore considérablement la capacité du modèle à détecter les fraudes (rappel élevé pour la classe 1).
  - Cependant, la faible précision montre que beaucoup de transactions légitimes sont mal classées comme frauduleuses.

- **Application pratique :**
  - Dans un système de détection de fraude, un rappel élevé est souvent prioritaire pour capturer un maximum de fraudes, même au prix de quelques faux positifs.
  - Pour améliorer la précision tout en conservant un bon rappel, il serait pertinent d'utiliser un modèle d'ensemble (comme un Random Forest) ou d'ajuster les seuils de décision.



### Conclusion

---

Dans cet exercice, nous avons appliqué la méthode **SMOTE (Synthetic Minority Oversampling Technique)** pour équilibrer les classes dans le dataset de détection de fraudes, en créant des échantillons synthétiques pour la classe minoritaire. Voici un résumé de nos observations et analyses :

---

#### Résumé des résultats
1. **Modèle avec les données originales :**
   - Ce modèle présente une excellente précision globale (**accuracy : 1.00**), mais il est fortement biaisé en faveur de la classe dominante (transactions légitimes).
   - Le rappel pour les fraudes est modéré (**0.63**), ce qui signifie que 37 % des fraudes passent inaperçues.
   - Cette approche est inadéquate dans un contexte où la détection des fraudes est critique, car elle manque de fiabilité pour capturer toutes les fraudes.

2. **Modèle avec les données SMOTE :**
   - Le rappel pour la classe fraude augmente considérablement à **0.92**, détectant presque toutes les transactions frauduleuses.
   - Cependant, la précision pour la classe fraude diminue à **0.06**, ce qui génère un grand nombre de faux positifs (transactions légitimes mal classées comme fraudes).
   - L'**accuracy globale** diminue légèrement à **0.97**, mais ce compromis est acceptable si l'objectif principal est de maximiser la détection des fraudes.

---

#### Interprétation des résultats
- **Avantages de SMOTE** :
  - SMOTE a permis d'équilibrer les données et d'améliorer la capacité du modèle à détecter les fraudes.
  - C'est une méthode efficace pour traiter le déséquilibre des classes dans des datasets où la classe minoritaire est cruciale.

- **Inconvénients de SMOTE** :
  - L'amélioration du rappel pour la classe fraude s'accompagne d'une baisse significative de la précision, ce qui peut poser problème dans des scénarios où les faux positifs doivent être minimisés.

---

#### Recommandations
1. **Utilisation dans un système réel :**
   - Si le rappel est prioritaire (par exemple, pour examiner manuellement les transactions détectées comme frauduleuses), SMOTE est une solution viable.
   - Cependant, pour améliorer la précision, il serait utile d'explorer d'autres approches, comme :
     - Ajuster les seuils de décision du modèle.
     - Utiliser des modèles d'ensemble (Random Forest, XGBoost).
     - Combiner SMOTE avec des techniques de sélection de caractéristiques pour éviter d'introduire du bruit.

2. **Pistes d'amélioration** :
   - Tester des algorithmes plus robustes pour le déséquilibre des classes (par exemple, **Balanced Random Forest** ou **AdaBoost**).
   - Comparer SMOTE avec d'autres techniques d'équilibrage, comme **ADASYN** ou **Borderline-SMOTE**.

---

En conclusion, SMOTE est une méthode puissante pour traiter le déséquilibre des classes et maximiser le rappel pour la détection des fraudes. Cependant, pour minimiser les faux positifs, des ajustements supplémentaires ou des modèles alternatifs doivent être envisagés.